In [ ]:
#### jefan 
#### begun: Dec 29 2016, updated: Jul 15 2018
#### analysis pipeline for "Results: Generalized object representations"

In [ ]:
from __future__ import division
import numpy as np

import matplotlib
import seaborn as sns
import prettyplotlib as ppl
from matplotlib import pylab, mlab, pyplot
from pylab import *
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
cm = matplotlib.cm
%matplotlib inline

import pandas as pd
import json
import sys
import os
import copy

#### set up paths 

In [ ]:
## define data directory (where you put downloaded data)
data_dir = './data'

## modify python path in order to load in useful variables
CURR_DIR = os.getcwd()

if os.path.join(CURR_DIR) not in sys.path:
    sys.path.append(CURR_DIR) 

#### globals

In [ ]:
## import list of 105 corresponding categories (between Imagenet and Eitz datasets)
from inet_sketch_cats import CATS
Cats = CATS

## standardized ordering of categories for ease of visualization
from inet_sketch_cats import STANDARD_ORDER
standard_order = STANDARD_ORDER

## standardized ordering of categories for ease of visualization
from inet_sketch_cats import SKLOOP_OBJS
skloop_objs = SKLOOP_OBJS

## load dictionary with correpsondences between Imagenet and Eitz labels
from inet_sketch_cats import INET_TO_SKETCH
inet_to_sketch = INET_TO_SKETCH

## load in dictionary with cluster assignments
from inet_sketch_cats import CLUSTER_OBJ_LIST
clusterObjList = CLUSTER_OBJ_LIST
clusterObjList = reduce(lambda x, y: list(x) + list(y), clusterObjList.values())

In [ ]:
## fix incongruities between original eitz and preferred spellings
meta_eitz = pd.read_csv(os.path.join(data_dir,'meta_eitz.csv'))
correspondence = dict([(x, x) for x in np.unique(meta_eitz['category'])])
correspondence['crane'] = 'crane (machine)'
correspondence['loudspeakers'] = 'loudspeaker'
correspondence['loudspeaker'] = 'loudspeakers'

### load in mean feature vectors for each class and generate RDM

In [ ]:
imF_means_layer1 = np.load(os.path.join(data_dir,'inet_class_means_layer1.npy'))
imF_means_fc6 = np.load(os.path.join(data_dir,'inet_class_means_fc6.npy'))

skF_means_layer1 = np.load(os.path.join(data_dir,'eitz_class_means_layer1.npy'))
skF_means_fc6 = np.load(os.path.join(data_dir,'eitz_class_means_fc6.npy'))

# load in other layers
skF_means_layer2 = np.load(os.path.join(data_dir,'eitz_class_means_layer2.npy'))
skF_means_layer3 = np.load(os.path.join(data_dir,'eitz_class_means_layer3.npy'))
skF_means_layer4 = np.load(os.path.join(data_dir,'eitz_class_means_layer4.npy'))
skF_means_layer5 = np.load(os.path.join(data_dir,'eitz_class_means_layer5.npy'))

imF_means_layer2 = np.load(os.path.join(data_dir,'inet_class_means_layer2.npy'))
imF_means_layer3 = np.load(os.path.join(data_dir,'inet_class_means_layer3.npy'))
imF_means_layer4 = np.load(os.path.join(data_dir,'inet_class_means_layer4.npy'))
imF_means_layer5 = np.load(os.path.join(data_dir,'inet_class_means_layer5.npy'))

assert imF_means_fc6.shape[0] == 105
assert skF_means_fc6.shape[0] == 105

In [ ]:
def plot_rdm(X,size):
    sns.set_style('white')
    sns.set_context('paper')
    fig = plt.figure(figsize=(size,size))
    fig.gca().matshow(X,cmap=cm.inferno)
    plt.xticks(range(len(standard_order)), standard_order, rotation=90);
    plt.yticks(range(len(standard_order)), standard_order); 

    for tick in pylab.gca().xaxis.iter_ticks():
        tick[0].label2On = True
        tick[0].label1On = False
        tick[0].label2.set_rotation('vertical')
        tick[0].tick1On = False
        tick[0].tick2On = False
    for tick in pylab.gca().yaxis.iter_ticks():
        tick[0].tick1On = False
        tick[0].tick2On = False
        
def plot_rdms_adjacent(X1,X2):
    '''
    X1 = feature means within each class for bottom layer
    X2 = feature means within each class for top layer    
    '''
    import seaborn as sns
    import matplotlib.cm as cm
    sns.set_style('white')
    sns.set_context('paper')
    fig = plt.figure(figsize=(24,12))
    
    plt.subplot(1,2,1)
    ax = fig.gca().matshow(X1,cmap=cm.inferno)
    plt.xticks(range(len(standard_order)), standard_order, rotation=90);
    plt.yticks(range(len(standard_order)), standard_order); 
    plt.colorbar(ax)

    for tick in pylab.gca().xaxis.iter_ticks():
        tick[0].label2On = True
        tick[0].label1On = False
        tick[0].label2.set_rotation('vertical')
        tick[0].tick1On = False
        tick[0].tick2On = False
    for tick in pylab.gca().yaxis.iter_ticks():
        tick[0].tick1On = False
        tick[0].tick2On = False    
        
    plt.subplot(1,2,2)       
    fig.gca().matshow(X2,cmap=cm.inferno)
    plt.xticks(range(len(standard_order)), standard_order, rotation=90);
    plt.yticks(range(len(standard_order)), standard_order); 

    for tick in pylab.gca().xaxis.iter_ticks():
        tick[0].label2On = True
        tick[0].label1On = False
        tick[0].label2.set_rotation('vertical')
        tick[0].tick1On = False
        tick[0].tick2On = False
    for tick in pylab.gca().yaxis.iter_ticks():
        tick[0].tick1On = False
        tick[0].tick2On = False     

In [ ]:
### Plot RDMs based on bottom ('layer1') and top ('fc6') layer feature representations for sketch domain (Eitz corpus)
sketch_rdm_layer1 = 1 - np.corrcoef(skF_means_layer1)
sketch_rdm_fc6 = 1 - np.corrcoef(skF_means_fc6)
plot_rdms_adjacent(sketch_rdm_layer1,sketch_rdm_fc6)

In [ ]:
### Plot RDMs based on bottom ('layer1') and top ('fc6') layer feature representations for photo domain (Imagenet)
photo_rdm_layer1 = 1 - np.corrcoef(imF_means_layer1)
photo_rdm_fc6 = 1 - np.corrcoef(imF_means_fc6)
plot_rdms_adjacent(photo_rdm_layer1,photo_rdm_fc6)

### Compute cross-domain RDM similarity ("second-order similarity")

In [ ]:
## compute cross-domain similarity
def get_rdm_similarity(a,b):
    '''
    "a" is one RDM made from, e.g., sketches
    "b" is another RDM made from, e.g., photos
    what is between-RDM similarity?
    '''    
    import scipy.spatial.distance as dist
    xdist = stats.pearsonr(dist.squareform(a, checks=False), dist.squareform(b, checks=False))    
    return xdist

In [ ]:
## set up pipeline to generate RDMs for all intermediate layers as well

def load_and_get_rdm(domain,layer,data_dir):
    if domain in ['photo','imagenet','inet']:
        d = 'inet'
    elif domain in ['sketch','eitz']:
        d = 'eitz'
    else:
        print 'Not valid domain!'
    means = np.load(os.path.join(data_dir,'{}_class_means_{}.npy'.format(d,layer)))
    rdm = 1 - np.corrcoef(means)
    return rdm
    
    
photo_rdm_layer2 = load_and_get_rdm('photo','layer2',data_dir)
photo_rdm_layer3 = load_and_get_rdm('photo','layer3',data_dir)
photo_rdm_layer4 = load_and_get_rdm('photo','layer4',data_dir)    
photo_rdm_layer5 = load_and_get_rdm('photo','layer5',data_dir)    

sketch_rdm_layer2 = load_and_get_rdm('eitz','layer2',data_dir)
sketch_rdm_layer3 = load_and_get_rdm('eitz','layer3',data_dir)
sketch_rdm_layer4 = load_and_get_rdm('eitz','layer4',data_dir)    
sketch_rdm_layer5 = load_and_get_rdm('eitz','layer5',data_dir) 

sketch_rdms = np.dstack((sketch_rdm_layer1,sketch_rdm_layer2,sketch_rdm_layer3,sketch_rdm_layer4,sketch_rdm_layer5,sketch_rdm_fc6))
photo_rdms = np.dstack((photo_rdm_layer1,photo_rdm_layer2,photo_rdm_layer3,photo_rdm_layer4,photo_rdm_layer5,photo_rdm_fc6))

In [ ]:
import scipy.stats as stats
dister = []
for l in range(sketch_rdms.shape[2]):
    r,p = get_rdm_similarity(sketch_rdms[:,:,l],photo_rdms[:,:,l])
    dister.append(r)

In [ ]:
dister = np.array(dister)    
print 'cross-domain similarity by layer'
print np.round(dister,4)

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(3,3))
sns.set_context('poster')
sns.set_style('white')
h = plt.plot(np.arange(len(dister))+1,dister, color=[0.2, 0.2, 0.2])
plt.xticks(np.arange(1,len(dister)+1))
h[0].set_color([0.2, 0.2, 0.2])
plt.xlabel('model layer')
plt.ylabel('photo-sketch similarity (r)')
plt.xlim([0.8,6.2])
plt.ylim([0,0.6])
if not os.path.exists('./plots'):
    os.makedirs('./plots')
plt.tight_layout()        
plt.savefig('./plots/1_crossdomain_similarity_by_layer.pdf')
plt.close(fig)

### First order comparison between feature representations across sketch and photo domains

In [ ]:
## Get direct measure of correspondence between sketch & photo feature vectors 
# (1) stack photo and sketch matrices; (2) Off-diagonal 105x105 super-block contains diagonal vector (105)
# with "direct" correlation between class means computed separately for each domain
stacked_fc6 = np.vstack((imF_means_fc6,skF_means_fc6))
stacked_mat = np.corrcoef(stacked_fc6)
## uncomment next line if you want to plot rdm for stacked fc6 feature mat
# plot_rdm(stacked_mat,24)
# get off-diagonal superblock
off_diag = stacked_mat[105:,:105]
# get diagonal of this superblock
corr_fc6 = np.diagonal(off_diag)
# get off-diagonal elements of this superblock
inds = np.triu_indices(np.shape(off_diag)[0],1)
off_diag_fc6 = off_diag[inds[0],inds[1]]
print 'fc6 features: sketch-photo correspondence | same_obj r = {}, diff_obj r = {}'.format(np.round(np.mean(corr_fc6),3),np.round(np.mean(off_diag_fc6),3))

In [ ]:
# do same for layer1 features
stacked_layer1 = np.vstack((imF_means_layer1,skF_means_layer1))
stacked_mat = np.corrcoef(stacked_layer1)
## uncomment next line if you want to plot stacked_mat for layer1
# plot_rdm(stacked_mat,24)
# get off-diagonal superblock
off_diag = stacked_mat[105:,:105]
# get diagonal of this superblock
corr_layer1 = np.diagonal(off_diag)
# get off-diagonal elements of this superblock
inds = np.triu_indices(np.shape(off_diag)[0],1)
off_diag_layer1 = off_diag[inds[0],inds[1]]
print 'layer1 features: sketch-photo correspondence | same_obj r = {}, diff_obj r = {}'.format(np.round(np.mean(corr_layer1),3),np.round(np.mean(off_diag_layer1),3))


In [ ]:
def correspondence_within_class_between_domains(im_means,sk_means):
    stacked = np.vstack((im_means,sk_means))
    stacked_mat = np.corrcoef(stacked)
    # get off-diagonal superblock
    off_diag = stacked_mat[105:,:105]
    # get diagonal of this superblock
    diag = np.diagonal(off_diag)
    # get off-diagonal elements of this superblock
    inds = np.triu_indices(np.shape(off_diag)[0],1)
    off_diag = off_diag[inds[0],inds[1]]
    return diag, off_diag

def get_boot_diff(x1,x2,nIter):
    boot_mean = []
    for i in range(nIter):
        boot1 = np.random.RandomState(i).choice(range(x1.shape[0]),size=x1.shape[0],replace=True)
        boot2 = np.random.RandomState(i).choice(range(x2.shape[0]),size=x2.shape[0],replace=True)
        boot_mean.append(np.mean(x1[boot1])-np.mean(x2[boot2]))
    p = sum(boot_mean<0)/nIter * 2
    return np.array(boot_mean), p

In [ ]:
# compute diag vs. off-diag for all layers
x1 = imF_means_layer1
x2 = skF_means_layer1
diag_layer1, off_diag_layer1 = correspondence_within_class_between_domains(x1,x2)
x1 = imF_means_layer2
x2 = skF_means_layer2
diag_layer2, off_diag_layer2 = correspondence_within_class_between_domains(x1,x2)
x1 = imF_means_layer3
x2 = skF_means_layer3
diag_layer3, off_diag_layer3 = correspondence_within_class_between_domains(x1,x2)
x1 = imF_means_layer4
x2 = skF_means_layer4
diag_layer4, off_diag_layer4 = correspondence_within_class_between_domains(x1,x2)
x1 = imF_means_layer5
x2 = skF_means_layer5
diag_layer5, off_diag_layer5 = correspondence_within_class_between_domains(x1,x2)
x1 = imF_means_fc6
x2 = skF_means_fc6
diag_fc6, off_diag_fc6 = correspondence_within_class_between_domains(x1,x2)

In [ ]:
# also plot first order similarity (correlations between sketch and photo of same object)
import seaborn as sns
fig = plt.figure(figsize=(3,3))
sns.set_context('poster')
sns.set_style('white')
diag = map(np.mean,[diag_layer1, diag_layer2, diag_layer3, diag_layer4, diag_layer5, diag_fc6])
h = plt.plot(np.arange(6)+1,diag, color=[0.2, 0.2, 0.2])
plt.xticks(np.arange(1,len(dister)+1))
h[0].set_color([0.2, 0.2, 0.2])
plt.xlabel('model layer')
plt.ylabel('photo-sketch first-order similarity (r)')
xx = plt.xlim([0.8,6.2])
yy = plt.ylim([0,0.3])
plt.close(fig)
if not os.path.exists('./plots'):
    os.makedirs('./plots')
plt.tight_layout()    
plt.savefig('./plots/1_crossdomain_first_order_similarity_by_layer.pdf')
plt.close(fig)

## Classification accuracy

In [ ]:
# import dldata.metrics.utils as utils # compute_metric method deprecated, moving onto sci-kit learn methods
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [ ]:
# load raw features and subset by 105 categories of interest shared across photo and sketch domains
meta = pd.read_csv(os.path.join(data_dir,'meta_eitz_250.csv'))  ## full 250 category list from Eitz paper
cats250 = meta['category']
inds = [i for i, j in enumerate(cats250) if j in CATS] # indices of original sketch set that are actually part of the correspondence
skF_feats_fc6 = np.load(os.path.join(data_dir,'eitz_features_fc6.npy')) ## loading in full fc6 feature matrix for Eitz dataset
feats = skF_feats_fc6[inds,:]
cats = cats250[inds]
assert (len(feats)==8320) & (len(cats)==len(feats))

In [ ]:
# set up cross validation split and get score for single cross-validation split
from sklearn import model_selection
from sklearn import svm
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    feats, cats, test_size=0.2, random_state=0)

# accuracy using SVM classifier with linear kernel
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
print clf.score(X_test, y_test)   

In [ ]:
## 5-fold crossval classification accuracy
import time
start = time.time()

## Running this will take a few minutes, so set the reallyRun flag to True if you really want to run.
reallyRun = False
if reallyRun:
    clf = svm.SVC(kernel='linear', C=1)
    scores = model_selection.cross_val_score(
       clf, feats, cats, cv=5)

    end = time.time()
    elapsed = end - start
    print "Time taken: ", elapsed, "seconds."  

    ## 'fc6' layer representation
    print scores
    print np.mean(scores), '% mean accuracy on 5 splits'
    print np.std(scores)
    
    ## plot recognition accuracy
    scores_fc6 = scores
    sns.set_style('white')
    sns.set_context('poster')
    textsize = 16
    fig = plt.figure(figsize=(1,6))
    ind = 0
    width = 0.8
    y = np.mean(scores_fc6)
    lb = np.percentile(scores_fc6,2.5)
    ub = np.percentile(scores_fc6,97.5)
    rect = ppl.bar(np.arange(1),[y],width,color=ppl.colors.set1,ecolor='k',xticklabels=['top'])
    rect = plt.errorbar(width/2,[y],yerr=[np.array(y-lb,ub-y)],ecolor='k')
    plt.ylabel('model recognition accuracy (cross-validated)',fontsize=textsize)
    plt.ylim([0,1.0])
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.tick_params(axis='both', which='minor', labelsize=18)    
    plt.tight_layout() 
    plt.savefig('./plots/1_fc6_sketch_classification_accuracy.pdf')
    plt.close(fig)
    